<a href="https://colab.research.google.com/github/slp22/data-engineering-project/blob/main/engineering_monkeypox_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Data Engineering | Pipeline

# Monkeypox Tweets

## Imports

In [1]:
import json
import logging
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import os, shutil, itertools
import pandas as pd
import pathlib as Path
import pickle
import PIL
import random
import seaborn as sns
import sklearn as sk
import warnings
import zipfile

from sqlite3 import connect
import time
from datetime import datetime
from dateutil.parser import parse
from dateutil.relativedelta import *
from dateutil.easter import *
from dateutil.rrule import *
from dateutil.parser import *
from datetime import *


### Google Drive

In [2]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

# https://colab.research.google.com/notebooks/snippets/sheets.ipynb#scrollTo=JiJVCmu3dhFa

# authorize access 
from google.colab import auth
auth.authenticate_user()

# read in from Google Sheets

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


Mounted at /content/drive


### Pyspark



In [ ]:
# # https://towardsdatascience.com/pyspark-on-google-colab-101-d31830b238be
# # https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
!tar -xf spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
!pip install -q findspark

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, col, lower
from pyspark.sql.types import StructType,StructField, StringType, IntegerType


## 1 | Pipeline Design


* **Business Problem:** Can we build a dashboard to monitor top trending topics on Twitter about monkeypox?
* **Data source:** [Kaggle Tweets on Monkeypox ](https://www.kaggle.com/datasets/thakurnirmalya/monkeypox2022tweets)
* **Impact Hypothesis:** 

## 2 | Data Ingestion

#### 1. [Twitter Dataset on the 2022 MonkeyPox Outbreak](https://www.kaggle.com/datasets/thakurnirmalya/monkeypox2022tweets) (Dataset is list of TweetIDs)

#### 2. [Twitter Hydrating](https://towardsdatascience.com/learn-how-to-easily-hydrate-tweets-a0f393ed340e#:~:text=Hydrating%20Tweets) with [DocNow Hydrator](https://github.com/DocNow/hydrator/releases)

#### 3. Import [hydrated tweets](https://drive.google.com/drive/folders/1NbddxuSF3v5YuOgjvA1G4WgfPUlKfiul?usp=sharing) from GoogleDrive to Colab

In [44]:
# worksheets = ['TweetIDs_Part1', 'TweetIDs_Part2', 'TweetIDs_Part3', 'TweetIDs_Part4', 'TweetIDs_Part5', 'TweetIDs_Part6']

# worksheet_1 = gc.open('TweetIDs_Part1').sheet1
worksheet_2 = gc.open('TweetIDs_Part2').sheet1
# worksheet_3 = gc.open('TweetIDs_Part3').sheet1
# worksheet_4 = gc.open('TweetIDs_Part4').sheet1
# worksheet_5 = gc.open('TweetIDs_Part5').sheet1
# worksheet_6 = gc.open('TweetIDs_Part6').sheet1

# get_all_values gives a list of rows
# rows_1 = worksheet_1.get_all_values()
rows_2 = worksheet_2.get_all_values()
# rows_3 = worksheet_3.get_all_values()
# rows_4 = worksheet_4.get_all_values()
# rows_5 = worksheet_5.get_all_values()
# rows_6 = worksheet_6.get_all_values()

# Convert to a DataFrame and render
# tweets_1 = pd.DataFrame.from_records(rows_1)
tweets_2 = pd.DataFrame.from_records(rows_2)
# tweets_3 = pd.DataFrame.from_records(rows_3)
# tweets_4 = pd.DataFrame.from_records(rows_4)
# tweets_5 = pd.DataFrame.from_records(rows_5)
# tweets_6 = pd.DataFrame.from_records(rows_6)

In [ ]:
print('tweets_1', tweets_1.shape)
print('tweets_2', tweets_2.shape)
print('tweets_3', tweets_3.shape)
print('tweets_4', tweets_4.shape)
print('tweets_5', tweets_5.shape)
print('tweets_6', tweets_6.shape)

tweets_1 (12656, 35)
tweets_2 (15294, 35)
tweets_3 (15140, 35)
tweets_4 (16874, 35)
tweets_5 (41280, 35)
tweets_6 (127941, 35)


In [ ]:
n = tweets_1.shape[0] + tweets_2.shape[0] + tweets_3.shape[0] + tweets_4.shape[0] + tweets_5.shape[0] + tweets_6.shape[0]
print('n =', n)

n = 229185


In [ ]:
tweets_1.head(2)

In [ ]:
tweets_1.columns = tweets_1.iloc[0]
tweets_1.head(2)

In [24]:
tweets_1 = tweets_1.drop([0])

In [ ]:
tweets_1.head(5)

## 3 | Exploratory Data Analysis

### Explore one set (tweet_1 n=12,656) 

In [ ]:
tweets_1.head(2)

In [ ]:
tweets_1[['text', 'user_created_at']]

### tweets_1

In [ ]:
tweets_1.info()

In [ ]:
tweets_1.describe()

In [ ]:
cols_list = list(tweets_1.columns)
cols_list

In [ ]:
# ['coordinates',
#  'created_at',
 'hashtags',
#  'media',
#  'urls',
#  'favorite_count',
#  'id',
#  'in_reply_to_screen_name',
#  'in_reply_to_status_id',
#  'in_reply_to_user_id',
#  'lang',
#  'place',
#  'possibly_sensitive',
#  'quote_id',
#  'retweet_count',
#  'retweet_id',
#  'retweet_screen_name',
#  'source',
 'text',
 'tweet_url',
 'user_created_at',
#  'user_id',
#  'user_default_profile_image',
#  'user_description',
#  'user_favourites_count',
#  'user_followers_count',
#  'user_friends_count',
#  'user_listed_count',
 'user_location',
#  'user_name',
 'user_screen_name',
#  'user_statuses_count',
#  'user_time_zone',
#  'user_urls',
#  'user_verified']

In [ ]:
tweets_1['hashtags']
tweets_1['hashtags'].unique()

In [ ]:
tweets_1['possibly_sensitive'][0:2]

In [ ]:
tweets_1['text']

In [ ]:
tweets_1['tweet_url'][0:2]

In [ ]:
tweets_1['user_created_at']

In [27]:
tweets_1['date'] = pd.to_datetime(tweets_1['user_created_at'], 
                                  format='%a %b %d %H:%M:%S +0000 %Y', 
                                  errors='coerce').dt.date

In [ ]:
tweets_1.info()

In [ ]:
tweets_1[['date']]

In [ ]:
tweets_1['user_id']

In [ ]:
tweets_1['user_location']

In [ ]:
tweets_1['user_screen_name']

In [31]:
tweets_1 = tweets_1[['date',
                     'user_screen_name',
                     'text',
                     'tweet_url',
                     'user_location',
                     'hashtags']]
tweets_1.head(2)

,date,user_screen_name,text,tweet_url,user_location,hashtags
2,2022-04-01,LeftistHank,oh monkey POX? I thought you said monkey POGS,https://twitter.com/LeftistHank/status/1527751...,"Chicago, IL",
6,2022-04-01,YUCCAYAWN,lemme be clear ... if you like your monkey pox...,https://twitter.com/YUCCAYAWN/status/152739626...,,


In [18]:
tweets_1 = tweets_1.sort_values('date')
tweets_1.head(2)

AttributeError: ignored

In [62]:
# https://stackoverflow.com/questions/22898824/filtering-pandas-dataframes-on-dates
# https://stackoverflow.com/questions/5619489/troubleshooting-descriptor-date-requires-a-datetime-datetime-object-but-rec

tweets_1 = tweets_1[(tweets_1['date'] > date(2022,1,1))] 
tweets_1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


IndexingError: ignored

In [ ]:
tweets_1.info()

In [63]:
tweets_1.reset_index(drop=True)

,date,user_screen_name,text,tweet_url,user_location,hashtags
0,2022-04-01,LeftistHank,oh monkey POX? I thought you said monkey POGS,https://twitter.com/LeftistHank/status/1527751...,"Chicago, IL",
1,2022-04-01,YUCCAYAWN,lemme be clear ... if you like your monkey pox...,https://twitter.com/YUCCAYAWN/status/152739626...,,
2,2022-04-01,pufpuffprincess,"@KIINGEFFIE_ @JayceBliss @Carol1mcs @TMZ Yes, ...",https://twitter.com/pufpuffprincess/status/152...,Charlie Lastra's arms,
3,2022-04-01,pufpuffprincess,@Craig4lyfe23 @NateSanti @Youngkid_Vibes @TMZ ...,https://twitter.com/pufpuffprincess/status/152...,Charlie Lastra's arms,
4,2022-04-01,Xenon1337xV2,Not happening. \n\nThe vast majority will beli...,https://twitter.com/Xenon1337xV2/status/152809...,,
...,...,...,...,...,...,...
1276,2022-05-18,ChickenStink,Twitter stop showing me pictures of the gross ...,https://twitter.com/ChickenStink/status/152706...,,NoPusMay
1277,2022-05-18,VoyagerERH,Me praying that monkey pox kills us all https:...,https://twitter.com/VoyagerERH/status/15273930...,,
1278,2022-05-18,Cletusrufus1984,@amlivemon That’s like the shingles of monkey pox,https://twitter.com/Cletusrufus1984/status/152...,,
1279,2022-05-18,DollyD2022,This is NO COINCIDENCE...... BILL GATES wants ...,https://twitter.com/DollyD2022/status/15273767...,,


In [66]:
tweets_1.to_csv('/content/drive/MyDrive/tweets_1_clean.csv')

### tweets_2

In [ ]:
tweets_2

In [49]:
tweets_2.columns = tweets_2.iloc[0]
tweets_2.head(2)

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified
0,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified
1,,Thu May 26 20:31:06 +0000 2022,,,,0,1529923099424178191,dustinbennett76,1529921422134497287,43171823,...,182,501,3,,Work in Progress,Kenny_Swift,8117,,,FALSE


In [50]:
tweets_2 = tweets_2.drop([0])

In [51]:
tweets_2['date'] = pd.to_datetime(tweets_2['user_created_at'], 
                                  format='%a %b %d %H:%M:%S +0000 %Y', 
                                  errors='coerce').dt.date

In [52]:
tweets_2 = tweets_2[['date',
                     'user_screen_name',
                     'text',
                     'tweet_url',
                     'user_location',
                     'hashtags']]
tweets_2.head(2)

,date,user_screen_name,text,tweet_url,user_location,hashtags
1,2011-04-01,Kenny_Swift,@dustinbennett76 @TheCarpatheonLM @SummaCumLou...,https://twitter.com/Kenny_Swift/status/1529923...,,
2,2022-04-01,biirSaniye,@OnlineAlison Just only focusing on the pictur...,https://twitter.com/biirSaniye/status/15285009...,,


In [53]:
tweets_2 = tweets_2[(tweets_2['date'] > date(2022,1,1))]


In [54]:
tweets_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1828 entries, 2 to 14825
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              1828 non-null   object
 1   user_screen_name  1828 non-null   object
 2   text              1828 non-null   object
 3   tweet_url         1828 non-null   object
 4   user_location     1828 non-null   object
 5   hashtags          1828 non-null   object
dtypes: object(6)
memory usage: 100.0+ KB


In [64]:
tweets_2.reset_index(drop=True)

,date,user_screen_name,text,tweet_url,user_location,hashtags
0,2022-04-01,biirSaniye,@OnlineAlison Just only focusing on the pictur...,https://twitter.com/biirSaniye/status/15285009...,,
1,2022-04-01,PasundaCom,Monkeypox &gt; Israel dan Swiss Negara Terbaru...,https://twitter.com/PasundaCom/status/15285205...,"DKI Jakarta, Indonesia",
2,2022-04-01,ALEXANDRAKOVITT,Fíjate la magia de las vacunas de nueva genera...,https://twitter.com/ALEXANDRAKOVITT/status/152...,,
3,2022-04-01,Deb0819,"We have a vaccine against Monkeypox, don't let...",https://twitter.com/Deb0819/status/15285122262...,"Santa Barbara, California",
4,2022-04-01,Deb0819,@CatPatrol19D60 I know for a fact Monkeypox ha...,https://twitter.com/Deb0819/status/15285059553...,"Santa Barbara, California",
...,...,...,...,...,...,...
1823,2022-05-18,BanAgenda21,#MonkeyPox\n#MoneyPox\n#PureBlood \n#Unvaxxed\...,https://twitter.com/BanAgenda21/status/1528161...,Free State of Florida,MonkeyPox MoneyPox PureBlood Unvaxxed ChrisSky...
1824,2022-05-25,JEmberrii,Monkeypox in cali now great what a year we are...,https://twitter.com/JEmberrii/status/152954592...,,
1825,2022-05-25,BanglaRelease,Over 200 cases of monkeypox worldwide\n\nhttps...,https://twitter.com/BanglaRelease/status/15299...,"London, Ontario",monkeypox
1826,2022-05-25,TanukiHollow,Yeh he got the wuko...monkey pox https://t.co/...,https://twitter.com/TanukiHollow/status/152992...,"New York, USA",


In [65]:
tweets_2.to_csv('/content/drive/MyDrive/tweets_2_clean.csv')

#### PySpark?

In [ ]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
# tweet_df = spark.read.csv('/content/tweets.csv', inferSchema=True, header='true')

spark_df = spark.createDataFrame(tweets_1)
spark_df.printSchema()
# spark_df.show()

root
 |-- coordinates: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- hashtags: string (nullable = true)
 |-- media: string (nullable = true)
 |-- urls: string (nullable = true)
 |-- favorite_count: string (nullable = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_screen_name: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id: string (nullable = true)
 |-- lang: string (nullable = true)
 |-- place: string (nullable = true)
 |-- possibly_sensitive: string (nullable = true)
 |-- quote_id: string (nullable = true)
 |-- retweet_count: string (nullable = true)
 |-- retweet_id: string (nullable = true)
 |-- retweet_screen_name: string (nullable = true)
 |-- source: string (nullable = true)
 |-- text: string (nullable = true)
 |-- tweet_url: string (nullable = true)
 |-- user_created_at: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_default_profile_image: string (nullable = true

In [ ]:
df_schema = StructType([StructField("date", StringType(), True),
                        StructField("tweet", StringType(), True)])


In [ ]:
tweets_1.select(col())

* RangeIndex: 6859 entries, 0 to 6858
* Data columns (total 36 columns)

### word count

In [ ]:
# type(df)
# df.head(2)
pandas_df = df[['date','tweet']]
pandas_df.head(2)

,date,tweet
0,2022-08-06,So has anyone begun to compile 'here's the sta...
1,2022-08-06,"Getting some groceries, topic shifted to covid..."


In [ ]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
df_schema = StructType([StructField("date", StringType(), True),
                        StructField("tweet", StringType(), True)])


In [ ]:
# tweet_df = spark.read.csv('/content/tweets.csv', inferSchema=True, header='true')

spark_df = spark.createDataFrame(pandas_df, schema=df_schema)
spark_df.printSchema()
spark_df.show()

root
 |-- date: string (nullable = true)
 |-- tweet: string (nullable = true)

+----------+--------------------+
|      date|               tweet|
+----------+--------------------+
|2022-08-06|So has anyone beg...|
|2022-08-06|Getting some groc...|
|2022-08-06|"Illinois Childre...|
|2022-08-06|Illinois daycare ...|
|2022-08-06|@Natrone86 @Junot...|
|2022-08-06|"CDC Recommends L...|
|2022-08-06|1,058,637 America...|
|2022-08-06|MF didn’t know sh...|
|2022-08-06|Three people of A...|
|2022-08-06|MonkeyPox Case: ರ...|
|2022-08-06|#Monkeypox Fallza...|
|2022-08-06|RT-PCR detection ...|
|2022-08-06|@BigBadDenis When...|
|2022-08-06|News Outlook TOP ...|
|2022-08-06|CDC Issues Eye-Op...|
|2022-08-06|United States dec...|
|2022-08-06|@FoxNews They are...|
|2022-08-06|@PetiteNicoco No....|
|2022-08-06|Idc if it’s Monke...|
|2022-08-06|In the technical ...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
spark_df = spark_df.withColumn('tweet', 
                               explode(split(lower(col('tweet')), '\s')))

In [ ]:
(spark_df.groupBy('tweet')
  .count()
  .orderBy('count', ascending=False)
  .show(40))

+----------+-----+
|     tweet|count|
+----------+-----+
|          | 4898|
|       the| 4351|
| monkeypox| 3230|
|        to| 2957|
|         a| 2400|
|       and| 2344|
|    monkey| 2151|
|        is| 2014|
|        of| 1901|
|         i| 1721|
|       pox| 1683|
|        in| 1666|
|       for| 1239|
|       you| 1083|
|      that| 1071|
|        it|  973|
|      with|  967|
|      this|  938|
|       not|  868|
|       are|  838|
|        be|  828|
|        on|  818|
|      have|  805|
|     about|  659|
|#monkeypox|  651|
|       get|  620|
|      from|  597|
|       but|  595|
|      they|  585|
|        we|  583|
|        so|  545|
|        my|  538|
|     covid|  528|
|        if|  520|
|        at|  511|
|      like|  509|
|      just|  489|
|        as|  480|
|    people|  471|
|        me|  454|
+----------+-----+
only showing top 40 rows



In [ ]:
(spark_df)

DataFrame[date: string, tweet: string]

###❓Count specific words using PySpark?[link text](https://)
* groupby, count, agg, 
* google = pyspark word frequency


### Corpus `tweets`

In [ ]:
tweets = df[['language','date','username','hashtags','tweet']]
tweets.head(2)

,language,date,username,hashtags,tweet
0,en,2022-08-06,thetenth2022,[],So has anyone begun to compile 'here's the sta...
1,en,2022-08-06,ashemedai,[],"Getting some groceries, topic shifted to covid..."


In [ ]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6859 entries, 0 to 6858
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   language  6859 non-null   object
 1   date      6859 non-null   object
 2   username  6859 non-null   object
 3   hashtags  6859 non-null   object
 4   tweet     6859 non-null   object
dtypes: object(5)
memory usage: 268.1+ KB


In [ ]:
tweets['username'].nunique()

6028

In [ ]:
# How many languages? 
tweets['language'].unique()

array(['en', 'kn', 'de', 'in', 'tl', 'fr', 'pt', 'te', 'tr', 'it', 'qme',
       'bn', 'qht', 'pl', 'es', 'el', 'nl', 'cy', 'ta', 'hi', 'sv', 'ja',
       'th', 'mr', 'et', 'gu', 'da', 'ro', 'ml', 'zxx', 'und', 'pa', 'ur',
       'ko', 'am', 'fi', 'zh', 'lt', 'hu', 'ru', 'ar', 'si'], dtype=object)

In [ ]:
# How many tweets in English?
print('English entries:', (tweets[tweets["language"] == 'en'].count())['language'])

# How many tweets in other languages?
print('Spanish entries:', (tweets[tweets["language"] == 'es'].count())['language'])
print('Italian entries:', (tweets[tweets["language"] == 'it'].count())['language'])

English entries: 6410
Spanish entries: 51
Italian entries: 10


In [ ]:
# Keep only English languge tweets
tweets = tweets[(tweets['language'] == 'en')]

In [ ]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6410 entries, 0 to 6858
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   language  6410 non-null   object
 1   date      6410 non-null   object
 2   username  6410 non-null   object
 3   hashtags  6410 non-null   object
 4   tweet     6410 non-null   object
dtypes: object(5)
memory usage: 300.5+ KB


In [ ]:
tweets.head(2)

,language,date,username,hashtags,tweet
0,en,2022-08-06,thetenth2022,[],So has anyone begun to compile 'here's the sta...
1,en,2022-08-06,ashemedai,[],"Getting some groceries, topic shifted to covid..."


In [ ]:
# Drop language column since we have only English tweets
tweets = tweets.drop(columns=['language'])

In [ ]:
tweets.head(10)

,date,username,hashtags,tweet
0,2022-08-06,thetenth2022,[],So has anyone begun to compile 'here's the sta...
1,2022-08-06,ashemedai,[],"Getting some groceries, topic shifted to covid..."
2,2022-08-06,democracymotion,[],"""Illinois Children's Daycare Worker Tests Posi..."
3,2022-08-06,thegoogle93,[],Illinois daycare worker tests positive for mon...
4,2022-08-06,bufflosouljah1,[],@Natrone86 @JunotIsrael @elcavaqueen @thechicc...
5,2022-08-06,democracymotion,[],"""CDC Recommends Limiting Sex Partners to Avoid..."
6,2022-08-06,salemjakes,[],"1,058,637 Americans have died in the U.S. Covi..."
7,2022-08-06,fineassbrei,[],MF didn’t know shit about monkey pox in June c...
8,2022-08-06,xenohadi,[],Three people of African origin still admitted ...
11,2022-08-06,geopoliticsind,[],RT-PCR detection kit for human monkeypox virus...


In [ ]:
# Save corpus
tweets.to_pickle('/content/tweets.pkl')
tweets.to_csv(r'/content/tweets.csv', index=False)

In [ ]:
# # copy corpus csv to Google Drive for Tableau
# shutil.copyfile('/content/tweets.csv', '/content/drive/MyDrive/tweets.csv')

# 4 | Storage

#### SQL Database `monkeypox.db`

##### helper functions

In [ ]:
# https://towardsdatascience.com/have-a-sql-interview-coming-up-ace-it-using-google-colab-6d3c0ffb29dc

def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    # # Setup local logging
    # logging.basicConfig(level=logging.INFO,
    #                     format='%(asctime)s %(levelname)s: %(message)s',
    #                     datefmt='%Y-%m-%d %H:%M:%S')

    # Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Connect to a DB file if it exists, else create a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    # logging.info(f'SQL DB {db_name} created')

    # Create table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    # logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Upload df
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""    
    cur.executemany(sql_string, rows_to_upload)
    # logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Commit the changes and close the connection
    con.commit()
    con.close()

In [ ]:
#  https://towardsdatascience.com/have-a-sql-interview-coming-up-ace-it-using-google-colab-6d3c0ffb29dc

def sql_query_to_pd(sql_query_string: str, db_name: str ='mpox.db') -> pd.DataFrame:
    
    # Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Close the connection
    con.close()

    # Return as df
    return pd.DataFrame(result_data, columns=cols)

##### sql_to_df

In [ ]:
# https://towardsdatascience.com/have-a-sql-interview-coming-up-ace-it-using-google-colab-6d3c0ffb29dc

# Read  csv as df
input_df = pd.read_csv('/content/tweets.csv')

# Upload df to a SQL table
pd_to_sqlDB(input_df,
            table_name='tweets',
            db_name='monkeypox.db')

# Write SQL query in a string variable
sql_query_string = """
    SELECT *
    FROM tweets
"""
# Exectue  SQL query
corpus = sql_query_to_pd(sql_query_string, db_name='monkeypox.db')
corpus

,date,username,hashtags,tweet
0,2022-08-06,thetenth2022,[],So has anyone begun to compile 'here's the sta...
1,2022-08-06,ashemedai,[],"Getting some groceries, topic shifted to covid..."
2,2022-08-06,democracymotion,[],"""Illinois Children's Daycare Worker Tests Posi..."
3,2022-08-06,thegoogle93,[],Illinois daycare worker tests positive for mon...
4,2022-08-06,bufflosouljah1,[],@Natrone86 @JunotIsrael @elcavaqueen @thechicc...
...,...,...,...,...
6405,2022-08-06,arkcowgirl62,[],"@hurtmeknots With Monkeypox on the rise, maybe..."
6406,2022-08-06,speehanagram,"['monkeypox', 'besafe']",@FITNESSSF can you start filling up the disinf...
6407,2022-08-06,pritzkertoilet,[],@politvidchannel 71% already have monkey pox .
6408,2022-08-06,thumbressler,['monkeypox'],RT if you have #monkeypox


* **database = `monkeypox.db`**
* **table_1 = `corpus` (6410 rows × 4 columns)**

In [ ]:
# # copy corpus csv to Google Drive for Tableau
# shutil.copyfile('/content/tweets.csv', '/content/drive/MyDrive/tweets.csv')

# 5 | Processing

### Case Counts by State

In [ ]:
# import sheet with state lattitude and longitude 
# data source: https://developers.google.com/public-data/docs/canonical/states_csv

# open google spreadsheet
worksheet = gc.open('USA-State-Coordinates').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

states = pd.DataFrame.from_records(rows)

states.columns = states.iloc[0]
states.drop([0], inplace=True)
states.drop(['state'], axis=1, inplace=True)
states.sort_values(by=['name'], inplace=True)
states = states.rename(columns={'latitude': 'lat', 'longitude': 'lon', 'name': 'state'})

states.tail(2)

,lat,lon,state
51,43.78444,-88.787868,Wisconsin
52,43.075968,-107.290284,Wyoming


In [ ]:
states.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 1 to 52
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lat     52 non-null     object
 1   lon     52 non-null     object
 2   state   52 non-null     object
dtypes: object(3)
memory usage: 1.6+ KB


In [ ]:
# import sheet with US case count by state 
# data source: https://www.cdc.gov/poxvirus/monkeypox/response/2022/us-map.html

worksheet = gc.open('2022-US-mpx-cases-by-state').sheet1
rows = worksheet.get_all_values()
cases = pd.DataFrame.from_records(rows)

cases.columns = cases.iloc[0]
cases.drop([0], inplace=True)
cases.drop(['AsOf', 'Case Range'], axis=1, inplace=True)
cases.sort_values(by=['Location'], inplace=True)

cases.tail(2)

,Location,Cases
51,Wisconsin,56
52,Wyoming,1


In [ ]:
# US case count by state (long + lat)
map_data = pd.concat([states, cases], axis=1)
map_data = map_data[['Location', 'Cases', 'lat', 'lon' ]]
map_data = map_data.rename(columns={'Location': 'state', 'Cases':'cases'})

map_data.tail(2)
# map_data.info()

,state,cases,lat,lon
51,Wisconsin,56,43.78444,-88.787868
52,Wyoming,1,43.075968,-107.290284


In [ ]:
map_data = map_data.astype({'cases':'int'})

map_data.sort_values(by=['cases'], ascending=False)
# map_data.sort_values(by=['latitude'], ascending=False)



,state,cases,lat,lon
5,California,3291,36.778261,-119.417932
33,New York,3124,43.299428,-74.217933
10,Florida,1739,27.664827,-81.515754
45,Texas,1472,31.968599,-99.901813
11,Georgia,1299,32.157435,-82.907123
14,Illinois,1005,40.633125,-89.398528
31,New Jersey,479,40.058324,-74.405661
39,Pennsylvania,477,41.203322,-77.194525
21,Maryland,461,39.045755,-76.641271
9,District of Columbia,414,38.905985,-77.033418


In [ ]:
map_data.head(2)

,state,cases,lat,lon
1,Alabama,53,32.318231,-86.902298
2,Alaska,3,63.588753,-154.493062


In [ ]:
map_data.to_csv('map_data.csv')  

# 6 | Deployment

See draft streamlit app here: https://slp22-data-engineering-project-streamlit-mpx-app-ckpzq2.streamlitapp.com/

Streamlit cheat sheet: https://daniellewisdl-streamlit-cheat-sheet-app-ytm9sg.streamlitapp.com/

In [ ]:
# wordcloud
# https://www.geeksforgeeks.org/generating-word-cloud-python/
comment_words = ''
stopwords = set(stopwords)
 
# iterate through the csv file
for val in df.text:
     
    # typecaste each val to string
    val = str(val)
 
    # split the value
    tokens = val.split()
     
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
     
    comment_words += " ".join(tokens)+" "
 
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(comment_words)
 
# plot the WordCloud image                      
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.title('Coronavirus Tweets April 2020')
plt.savefig("coronavirus-tweets-word-cloud.jpeg");

# 7 | Testing/Robustness

[Python schedule](https://schedule.readthedocs.io/en/stable/examples.html#run-a-job-every-x-minute)